# Multi Class Classification and generalization
In this tutorial, we demonstrate the power of single qubit to perform multi class classification on MNIST dataset using a hybrid model. We also show the generalization in QML with few training data.

In [1]:
from importlib.util import find_spec
%load_ext autoreload
%autoreload 2

%matplotlib inline

if find_spec("qml_hep_lhc") is None:
    import sys
    sys.path.append('../..')

In [2]:
from qml_hep_lhc.data import ElectronPhoton, MNIST, QuarkGluon
from qml_hep_lhc.models import QCNNHybrid
from tensorflow.keras.callbacks import ReduceLROnPlateau
import argparse
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt
import time

In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [4]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPU


## Single qubit Hybrid model

In [ ]:
args = argparse.Namespace()

# Data
args.resize = [8,8]
args.standardize = 1
args.batch_size = 128
args.validation_split = 0.05
args.labels_to_categorical = 1
args.percent_samples = 0.5
args.opt = 'Ranger'

# Base Model
args.learning_rate = 1e-3
args.epochs = 10

# Quantum CNN Parameters
args.n_layers = 1
args.ansatz = "NQubit"
args.num_qconv_layers = 1
args.qconv_dims = [1]
args.num_fc_layers = 1
args.fc_dims = [8]

In [54]:
data = MNIST(args)
data.prepare_data()
data.setup()
print(data)

Resizing data...
Resizing data...
Standardizing data...
Converting labels to categorical...
Converting labels to categorical...

Dataset :MNIST
╒════════╤══════════════════╤═════════════════╤═════════════════╤═══════════╕
│ Data   │ Train size       │ Val size        │ Test size       │ Dims      │
╞════════╪══════════════════╪═════════════════╪═════════════════╪═══════════╡
│ X      │ (28500, 8, 8, 1) │ (1500, 8, 8, 1) │ (5000, 8, 8, 1) │ (8, 8, 1) │
├────────┼──────────────────┼─────────────────┼─────────────────┼───────────┤
│ y      │ (28500, 10)      │ (1500, 10)      │ (5000, 10)      │ (10,)     │
╘════════╧══════════════════╧═════════════════╧═════════════════╧═══════════╛

╒══════════════╤═══════╤════════╤════════╤═══════╤══════════════════════════════════════════════════════════════╕
│ Type         │   Min │    Max │   Mean │   Std │ Samples for each class                                       │
╞══════════════╪═══════╪════════╪════════╪═══════╪═══════════════════════════════

In [55]:
data.config()

{'input_dims': (8, 8, 1),
 'output_dims': (10,),
 'mapping': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}

In [56]:
model = QCNNHybrid(data.config(), args)

../../qml_hep_lhc/layers/qconv2d.py:65: UserWarning: fm_class is only used for TwoLayerPQC.
  warnings.warn('fm_class is only used for TwoLayerPQC.')
../../qml_hep_lhc/layers/qconv2d.py:69: UserWarning: ansatz_class is only used for TwoLayerPQC.
  warnings.warn('ansatz_class is only used for TwoLayerPQC.')


In [57]:
model.compile()
model.build_graph().summary()

Model: "QCNNHybrid-AngleMap-NQubit"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 8, 8, 1)]         0         
                                                                 
 qconv2d_0 (QConv2D)         (None, 6, 6, 1, 1)        18        
                                                                 
 flatten_4494 (Flatten)      (None, 36)                0         
                                                                 
 dense_14 (Dense)            (None, 8)                 296       
                                                                 
 dense_15 (Dense)            (None, 10)                90        
                                                                 
Total params: 404
Trainable params: 404
Non-trainable params: 0
_________________________________________________________________


In [58]:
lr_scheduler_callback = ReduceLROnPlateau(monitor='val_loss',
                                              factor=np.sqrt(0.1),
                                              patience=5,
                                              min_delta=0.0001,
                                              min_lr=1e-8)
callbacks = [lr_scheduler_callback]

In [59]:
model.fit(data, callbacks)

Epoch 1/10
223/223 [==============================] - 98s 441ms/step - loss: 2.2654 - auc_7: 0.5927 - accuracy: 0.1711 - val_loss: 2.1058 - val_auc_7: 0.7112 - val_accuracy: 0.2887 - lr: 0.0010
Epoch 2/10
223/223 [==============================] - 97s 436ms/step - loss: 1.7814 - auc_7: 0.8215 - accuracy: 0.4104 - val_loss: 1.4940 - val_auc_7: 0.8850 - val_accuracy: 0.5140 - lr: 0.0010
Epoch 3/10
223/223 [==============================] - 100s 447ms/step - loss: 1.2785 - auc_7: 0.9160 - accuracy: 0.5954 - val_loss: 1.1324 - val_auc_7: 0.9339 - val_accuracy: 0.6240 - lr: 0.0010
Epoch 4/10
223/223 [==============================] - 96s 431ms/step - loss: 1.0256 - auc_7: 0.9449 - accuracy: 0.6747 - val_loss: 0.9733 - val_auc_7: 0.9500 - val_accuracy: 0.6780 - lr: 0.0010
Epoch 5/10
223/223 [==============================] - 96s 430ms/step - loss: 0.9076 - auc_7: 0.9555 - accuracy: 0.7097 - val_loss: 0.8902 - val_auc_7: 0.9568 - val_accuracy: 0.7113 - lr: 0.0010
Epoch 6/10
223/223 [=========

In [60]:
model.test(data, callbacks=callbacks)

40/40 [==============================] - 11s 269ms/step - loss: 0.8021 - auc_7: 0.9623 - accuracy: 0.7370


[0.8021469712257385, 0.9622802734375, 0.7369999885559082, 0.9622802734375]

## Training with few data points
From the above training results we see that the model generalizes well on test set. Here we train the model with few 100 points per class. 

In [91]:
args = argparse.Namespace()

# Data
args.center_crop = 0.7
args.resize = [8,8]
args.normalize = 1
args.batch_size = 16
args.validation_split = 0.05
args.labels_to_categorical = 1
args.percent_samples = 0.02
args.opt = 'Ranger'

# Base Model
args.learning_rate = 1e-2
args.epochs = 10

# Quantum CNN Parameters
args.n_layers = 2
args.ansatz = "NQubit"
args.num_qconv_layers = 1
args.qconv_dims = [1]
args.num_fc_layers = 1
args.fc_dims = [8]

In [92]:
data = MNIST(args)
data.prepare_data()
data.setup()
print(data)

Center cropping...
Center cropping...
Resizing data...
Resizing data...
Normalizing data...
Converting labels to categorical...
Converting labels to categorical...

Dataset :MNIST
╒════════╤═════════════════╤═══════════════╤════════════════╤═══════════╕
│ Data   │ Train size      │ Val size      │ Test size      │ Dims      │
╞════════╪═════════════════╪═══════════════╪════════════════╪═══════════╡
│ X      │ (1140, 8, 8, 1) │ (60, 8, 8, 1) │ (200, 8, 8, 1) │ (8, 8, 1) │
├────────┼─────────────────┼───────────────┼────────────────┼───────────┤
│ y      │ (1140, 10)      │ (60, 10)      │ (200, 10)      │ (10,)     │
╘════════╧═════════════════╧═══════════════╧════════════════╧═══════════╛

╒══════════════╤═══════╤═══════╤════════╤═══════╤════════════════════════════════════════════════════╕
│ Type         │   Min │   Max │   Mean │   Std │ Samples for each class                             │
╞══════════════╪═══════╪═══════╪════════╪═══════╪══════════════════════════════════════════════

In [93]:
lr_scheduler_callback = ReduceLROnPlateau(monitor='val_loss',
                                              factor=np.sqrt(0.1),
                                              patience=5,
                                              min_delta=0.001,
                                              min_lr=1e-8)
callbacks = [lr_scheduler_callback]
model = QCNNHybrid(data.config(), args)
model.compile()
model.build_graph().summary()

Model: "QCNNHybrid-AngleMap-NQubit"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_17 (InputLayer)       [(None, 8, 8, 1)]         0         
                                                                 
 qconv2d_0 (QConv2D)         (None, 6, 6, 1, 1)        36        
                                                                 
 flatten_18180 (Flatten)     (None, 36)                0         
                                                                 
 dense_32 (Dense)            (None, 8)                 296       
                                                                 
 dense_33 (Dense)            (None, 10)                90        
                                                                 
Total params: 422
Trainable params: 422
Non-trainable params: 0
_________________________________________________________________


../../qml_hep_lhc/layers/qconv2d.py:65: UserWarning: fm_class is only used for TwoLayerPQC.
  warnings.warn('fm_class is only used for TwoLayerPQC.')
../../qml_hep_lhc/layers/qconv2d.py:69: UserWarning: ansatz_class is only used for TwoLayerPQC.
  warnings.warn('ansatz_class is only used for TwoLayerPQC.')


In [94]:
model.fit(data, callbacks)

Epoch 1/10
72/72 [==============================] - 8s 111ms/step - loss: 1.8194 - auc_16: 0.8014 - accuracy: 0.3816 - val_loss: 1.3594 - val_auc_16: 0.9023 - val_accuracy: 0.6167 - lr: 0.0100
Epoch 2/10
72/72 [==============================] - 8s 110ms/step - loss: 0.9353 - auc_16: 0.9528 - accuracy: 0.6877 - val_loss: 1.1194 - val_auc_16: 0.9289 - val_accuracy: 0.6333 - lr: 0.0100
Epoch 3/10
72/72 [==============================] - 8s 110ms/step - loss: 0.7100 - auc_16: 0.9701 - accuracy: 0.7719 - val_loss: 1.0006 - val_auc_16: 0.9437 - val_accuracy: 0.6000 - lr: 0.0100
Epoch 4/10
72/72 [==============================] - 8s 111ms/step - loss: 0.5966 - auc_16: 0.9778 - accuracy: 0.8184 - val_loss: 1.0114 - val_auc_16: 0.9412 - val_accuracy: 0.6333 - lr: 0.0100
Epoch 5/10
72/72 [==============================] - 8s 111ms/step - loss: 0.5814 - auc_16: 0.9781 - accuracy: 0.8202 - val_loss: 0.9918 - val_auc_16: 0.9429 - val_accuracy: 0.6667 - lr: 0.0100
Epoch 6/10
72/72 [=================

In [95]:
model.test(data, callbacks=callbacks)

13/13 [==============================] - 1s 72ms/step - loss: 0.5601 - auc_16: 0.9790 - accuracy: 0.8100


[0.5600734949111938,
 0.9790485501289368,
 0.8100000023841858,
 0.9790485501289368]

### Testing with more data points
The model generalizes on 200 test data points. We now test it with 10,000 followed by 57,000 test data points.

In [98]:
args = argparse.Namespace()

# Data
args.center_crop = 0.7
args.resize = [8,8]
args.normalize = 1
args.batch_size = 256
args.validation_split = 0.05
args.labels_to_categorical = 1
args.opt = 'Ranger'

data = MNIST(args)
data.prepare_data()
data.setup()
print(data)

Center cropping...
Center cropping...
Resizing data...
Resizing data...
Normalizing data...
Converting labels to categorical...
Converting labels to categorical...

Dataset :MNIST
╒════════╤══════════════════╤═════════════════╤══════════════════╤═══════════╕
│ Data   │ Train size       │ Val size        │ Test size        │ Dims      │
╞════════╪══════════════════╪═════════════════╪══════════════════╪═══════════╡
│ X      │ (57000, 8, 8, 1) │ (3000, 8, 8, 1) │ (10000, 8, 8, 1) │ (8, 8, 1) │
├────────┼──────────────────┼─────────────────┼──────────────────┼───────────┤
│ y      │ (57000, 10)      │ (3000, 10)      │ (10000, 10)      │ (10,)     │
╘════════╧══════════════════╧═════════════════╧══════════════════╧═══════════╛

╒══════════════╤═══════╤═══════╤════════╤═══════╤══════════════════════════════════════════════════════════════╕
│ Type         │   Min │   Max │   Mean │   Std │ Samples for each class                                       │
╞══════════════╪═══════╪═══════╪════════

In [99]:
model.test(data, callbacks=callbacks)

40/40 [==============================] - 38s 936ms/step - loss: 0.5731 - auc_16: 0.9774 - accuracy: 0.8227


[0.5730593800544739, 0.9774295687675476, 0.822700023651123, 0.9774295687675476]

In [100]:
data.test_ds = data.train_ds

In [101]:
model.test(data, callbacks=callbacks)

223/223 [==============================] - 213s 954ms/step - loss: 0.5776 - auc_16: 0.9773 - accuracy: 0.8222


[0.5776097178459167,
 0.9773194193840027,
 0.8222105503082275,
 0.9773194193840027]

We can conclude that the model generalizes well on the high test data points. The takeaway is that the QCNN has learn with few training data.